# Speeding Up Your Python Codes 1000x

## Introduction

Welcome to "Speeding Up Your Python Codes 1000x"!
In the next 45 minutes, we will learn tricks to unlock python performance and increase a sample code's speed by a factor of 1000x!

## The $n$-body Problem and the Leapfrog Algorithm

The $n$-body problem involves simulating the motion of many bodies interacting with each other through (usually) gravitational forces.
Each body experiences a force from every other body, making the simulation computationally intensive---ideal for exploring optimization techniques.

The leapfrog algorithm is a simple, robust numerical integrator that alternates between updating velocities ("kicks") and positions ("drifts").
It is popular because it conserves energy and momentum well over long simulation periods.

### Gravitational Force/Acceleration Equation

For any two bodies labeled by $i$ and $j$, the direct gravitational force exerted on body $i$ by body $j$ is given by:
\begin{align}
  \mathbf{f}_{ij} = -G m_i m_j\frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^{3/2}},
\end{align}
where:
* $G$ is the gravitational constant.
* $m_i$ and $m_j$ are the masses of the bodies.
* $\mathbf{r}_i$ and $\mathbf{r}_j$ are their position vectors.

Summing the contributions from all other bodies gives the net force (and thus the acceleration) on body $i$:
\begin{align}
  \mathbf{f}_i 
  = \sum_{j\ne i} \mathbf{f}_{ij} 
  = -G m_i \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3},
\end{align}

Given Newton's law $f = m a$, the "acceleration" applied on body $i$ caused by all other bodies is:
\begin{align}
  \mathbf{a}_i 
  = \sum_{j\ne i} \mathbf{a}_{ij} 
  = -G \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3}.
\end{align}

Choosing the right units so $G = 1$.
Here is a pure Python implementation of the gravitational acceleration:

In [ ]:
def acc1(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                axij = - m[j] * (xi - xj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                ayij = - m[j] * (yi - yj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                azij = - m[j] * (zi - zj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)

                axi += axij
                ayi += ayij
                azi += azij

        a.append((axi, ayi, azi))

    return a

We may try using this function to evaluate the gravitational force between two particles, with with mass $m = 1$, at a distance of $2$.
We expect the "acceleration" be -1/4 in the direction of their seperation.

In [ ]:
m = [1.0, 1.0]
r = [
    (+1.0, 0.0, 0.0),
    (-1.0, 0.0, 0.0),
]
aref = [
    (-0.25, 0.0, 0.0),
    (+0.25, 0.0, 0.0),
]

In [ ]:
a = acc1(m, r)

In [ ]:
a

In [ ]:
assert acc1(m, r) == aref

## Leapfrog Integrator

The leapfrog algorithm is a time-stepping method used to integrate the equations of motion.
It is especially favored for problems like the $n$-body simulation because it is symplectic, meaning it conserves energy and momentum over long periods.
The algorithm updates positions and velocities in a staggered manner:

1. Half-step velocity update (Kick):
   start by updating the velocity by half a time step using the current acceleration $a(t)$:
   \begin{align}
     v\left(t+\frac{1}{2}\Delta t\right) = v(t) + \frac{1}{2}\Delta t\, a(t).
   \end{align}

2. Full-step position update (Drift):
   update the position for a full time step using the half-stepped velocity:
   \begin{align}
     x(t+\Delta t) = x(t) + \Delta t\, v\left(t+\frac{1}{2}\Delta t\right).
   \end{align}

3. Half-step velocity update (Kick):
   finally, update the velocity by another half time step using the new acceleration $a(t+\Delta t)$ computed from the updated positions:
   \begin{align}
     v(t+\Delta t) = v\left(t+\frac{1}{2}\Delta t\right) + \frac{1}{2}\Delta t\, a(t+\Delta t).
   \end{align}

In [ ]:
def leapfrog(m, r0, v0, dt, acc=acc1):

    n  = len(m)

    # vh = v0 + 0.5 * dt * a0
    a0 = acc(m, r0)
    vh = []
    for i in range(n):
        a0xi, a0yi, a0zi = a0[i]
        v0xi, v0yi, v0zi = v0[i]
        vhxi = v0xi + 0.5 * dt * a0xi
        vhyi = v0yi + 0.5 * dt * a0yi
        vhzi = v0zi + 0.5 * dt * a0zi
        vh.append((vhxi, vhyi, vhzi))

    # r1 = r0 + dt * vh
    r1 = []
    for i in range(n):
        vhxi, vhyi, vhzi = vh[i]
        x0i,  y0i,  z0i  = r0[i]
        x1i = x0i + dt * vhxi
        y1i = y0i + dt * vhyi
        z1i = z0i + dt * vhzi
        r1.append((x1i, y1i, z1i))

    # v1 = vh + 0.5 * dt * a1
    a1 = acc(m, r1)
    v1 = []
    for i in range(n):
        a1xi, a1yi, a1zi = a1[i]
        vhxi, vhyi, vhzi = vh[i]
        v1xi = vhxi + 0.5 * dt * a1xi
        v1yi = vhyi + 0.5 * dt * a1yi
        v1zi = vhzi + 0.5 * dt * a1zi
        v1.append((v1xi, v1yi, v1zi))

    return r1, v1